This script collects the data from the initial CSV files provided by Storj and maps it to specific users. It then aggregates a number of features by user to be used as a basis for clustering analysis. It outputs the file 'userDataFree.pkl'. -- Lisa Marshall


**The following cells grab the raw data. To pull in somewhat pre-prepped versions for faster start, scroll down to 'Quickstart' section**

In [ ]:
import pandas as pd, numpy as np
import datetime as dt
##Pull in data provided by Storj in csv files
apNodes = pd.read_csv('data/ap_nodes_a.csv')
euNodes = pd.read_csv('data/eu_nodes_a.csv')
usNodes = pd.read_csv('data/us_nodes_a.csv')
paymentsap = pd.read_csv('data/ap_node_payments.csv')
paymentseu = pd.read_csv('data/eu_node_payments.csv')
paymentsus = pd.read_csv('data/us_node_payments.csv')
projectsap = pd.read_csv('data/ap_projects.csv')
projectseu = pd.read_csv('data/eu_projects.csv')
projectsus = pd.read_csv('data/us_projects.csv')
payReceiptap = pd.read_csv('data/ap_node_payments_receipt.csv')
payReceipteu = pd.read_csv('data/eu_node_payments_receipt.csv')
payReceiptus = pd.read_csv('data/us_node_payments_receipt.csv')
usersap = pd.read_csv('data/ap_users_a.csv')
userseu = pd.read_csv('data/eu_users_a.csv')
usersus = pd.read_csv('data/us_users_a.csv')

In [3]:
stripe_us = pd.read_csv('stripe_us.csv')
stripe_ap = pd.read_csv('stripe_ap.csv')
stripe_eu = pd.read_csv('stripe_eu.csv')
allStripes = stripe_us.append(stripe_ap).append(stripe_eu)

In [5]:
allStripes.to_csv('allPayers.csv')

In [ ]:
#Combine the regions into single sets
allNodes = apNodes.append(euNodes).append(usNodes)
usersAll = usersap.append(userseu).append(usersus)
paymentsAll = paymentsap.append(paymentseu).append(paymentsus)
projectsAll = projectsap.append(projectseu).append(projectsus)
payReceiptsAll = payReceiptap.append(payReceipteu).append(payReceiptus)

In [ ]:
allNodes = allNodes.iloc[:,1:].drop_duplicates()
usersAll = usersAll.iloc[:,1:].drop_duplicates()

In [ ]:
#Lowercase wallet addresses, node ids, and email hashes, etc
allNodes['wallet'] = allNodes['wallet'].astype(str).str.lower()
allNodes['id'] = allNodes['id'].astype(str).str.lower()
allNodes['email_hash'] = allNodes['email_hash'].astype(str).str.lower()
usersAll['email_hash'] = usersAll['email_hash'].astype(str).str.lower()
usersAll['id'] = usersAll['id'].astype(str).str.lower()
usersAll['password_hash'] = usersAll['password_hash'].astype(str).str.lower()
paymentsAll['node_id'] = paymentsAll['node_id'].astype(str).str.lower()
projectsAll['project_id'] = projectsAll['project_id'].astype(str).str.lower()
projectsAll['user_id'] = projectsAll['user_id'].astype(str).str.lower()
payReceiptsAll['node_id'] = payReceiptsAll['node_id'].astype(str).str.lower()
payReceiptsAll['receipt'] = payReceiptsAll['receipt'].astype(str).str.lower()

In [ ]:
#Convert payment period to datetime and convert amount (Ben indicated correct amt is amount/1000000)
payReceiptsAll.period = pd.to_datetime(payReceiptsAll.period).dt.date
payReceiptsAll['adjustedAmount'] = payReceiptsAll['amount']/1000000
paymentsAll.period = pd.to_datetime(paymentsAll.period).dt.date
paymentsAll['adjustedAmount'] = paymentsAll['amount']/1000000
allNodes.created_at = pd.to_datetime(allNodes.created_at).dt.date
allNodes.vetted_at = pd.to_datetime(allNodes.vetted_at).dt.date
allNodes.last_contact_success = pd.to_datetime(allNodes.last_contact_success).dt.date
allNodes.last_contact_failure = pd.to_datetime(allNodes.last_contact_failure).dt.date
allNodes.disqualified = pd.to_datetime(allNodes.disqualified).dt.date
allNodes.exit_finished_at = pd.to_datetime(allNodes.exit_finished_at).dt.date
allNodes.suspended = pd.to_datetime(allNodes.suspended).dt.date
usersAll.created_at = pd.to_datetime(usersAll.created_at).dt.date
projectsAll.created_at = pd.to_datetime(projectsAll.created_at).dt.date

In [ ]:
allNodes = allNodes.drop_duplicates()
usersAll = usersAll.drop_duplicates()
paymentsAll = paymentsAll.drop_duplicates()
payReceiptsAll = payReceiptsAll.drop_duplicates()
projectsAll = projectsAll.drop_duplicates()

In [ ]:
#Save standardized versions of data to file
'''
allNodes.to_csv('allNodes.csv')
usersAll.to_csv('allUsers.csv')
paymentsAll.to_csv('allPayments.csv')
payReceiptsAll.to_csv('allPayReceipts.csv')
projectsAll.to_csv('allProjects.csv')
'''

In [ ]:
#Get bucket data -- the per-project data
ap_bbr = pd.read_csv('bucket_data/ap_bbr.csv')
eu_bbr = pd.read_csv('bucket_data/eu_bbr.csv')
us_bbr = pd.read_csv('bucket_data/us_bbr.csv')
ap_bst = pd.read_csv('bucket_data/ap_bst.csv')
eu_bst = pd.read_csv('bucket_data/eu_bst.csv')
us_bst = pd.read_csv('bucket_data/us_bst.csv')

In [ ]:
#bbr = bandwidth usage for project, measured by bucket (a project can have multiple buckets)
#multiple entries for one project in same time period will be for different buckets
#action = upload(1) or download(2)
#inline = not stored on nodes
#settled = total stored, including on nodes
bbr = ap_bbr.append(eu_bbr).append(us_bbr)
bbr = bbr.iloc[:,1:]
bbr['project_id'] = bbr['project_id'].astype(str).str.lower()
bbr.interval_start = pd.to_datetime(bbr.interval_start)
bbr = bbr.drop_duplicates()
#bbr.to_csv('project_bandwidth.csv')

#bst = number of bytes stored per project (by bucket) per time interval
bst = ap_bst.append(eu_bst).append(us_bst)
bst = bst.iloc[:,1:]
bst['project_id'] = bst['project_id'].astype(str).str.lower()
bst.interval_start = pd.to_datetime(bst.interval_start)
bbr = bbr.drop_duplicates()
#bst.to_csv('project_bytes_stored.csv')


**Quickstart: Upload pre-prepped files with cell below**

In [2]:
#upload full set
import pandas as pd, numpy as np
import datetime as dt
allNodes = pd.read_csv('allNodes.csv')
usersAll = pd.read_csv('allUsers.csv')
paymentsAll = pd.read_csv('allPayments.csv')
payReceiptsAll = pd.read_csv('allPayReceipts.csv')
projectsAll = pd.read_csv('allProjects.csv')
bbr = pd.read_csv('project_bandwidth.csv')
bst = pd.read_csv('project_bytes_stored.csv')
##upload tier 1 transaction data too

In [2]:
#Convert bytes to GB
projectsAll.bandwidth_limit = projectsAll.bandwidth_limit/1000000000

In [3]:
#Search for nodes with 2070 as last contact success year (= 1970, ie, there's never been contacted)
#This will actually be used further down in the code.
tester = allNodes[allNodes.last_contact_success=='2070-01-01']

In [4]:
tester

,Unnamed: 0,id,audit_success_count,total_audit_count,uptime_success_count,total_uptime_count,created_at,vetted_at,wallet,free_disk,last_contact_success,last_contact_failure,contained,disqualified,suspended,piece_count,audit_reputation_alpha,audit_reputation_beta,exit_finished_at,email_hash
74880,5225,23ae3c4aec2880e1c4b39c036cd8c8a826e9e9b2571312...,0,0,3344,17540,2019-04-02,NaN,0x57f6a770064a5d4cfefbde66e0891699dc0dd426,1700000000000,2070-01-01,2020-12-16,False,2021-04-06,NaN,0,50.0,0.0,NaN,f34912a1d0fcc21b27e4c2a11adc67d4fa5de69b5f9d4f...
86121,17996,7c47061e8a5e53fe9a26ce6ea6250f18c000ed862c5290...,0,0,1637,16796,2019-04-02,NaN,0x1aadce023498f6d9e23230d86039f93f9dff4807,-2132077848,2070-01-01,2020-12-16,False,2021-04-06,NaN,0,50.0,0.0,NaN,f34912a1d0fcc21b27e4c2a11adc67d4fa5de69b5f9d4f...
100206,33920,e99cefccf01852dde6e2a459cf51cc5f295b16acc21a96...,0,0,2189,16256,2019-04-02,NaN,0x0000000000000000000000000000000000000000,1000000000000,2070-01-01,2020-12-16,False,2021-04-06,NaN,0,50.0,0.0,NaN,f34912a1d0fcc21b27e4c2a11adc67d4fa5de69b5f9d4f...


In [5]:
#Make sure these nodes have never received payment
paymentsAll[paymentsAll.node_id.isin(list(tester.id))]

,Unnamed: 0,period,node_id,amount,adjustedAmount
279058,91,2019-03-01,23ae3c4aec2880e1c4b39c036cd8c8a826e9e9b2571312...,0,0.0
279566,599,2019-04-01,23ae3c4aec2880e1c4b39c036cd8c8a826e9e9b2571312...,0,0.0


**Since these nodes, and the 0x0000... node have never received payment (and apparently have never been operational), I suggest we remove them from the dataset. (I've retained them in the full set download but removed them in the 'reduced' set)**

In [6]:
#mark project as free or paid tier
#Ben explained in Slack: 'free accounts cannot exceed more than 150 GB-month or 150gb egress 
#per month. paid accounts can exceed these limits. the 150gb is arrived at by there being 50gb 
#limit per project with a 3 project limit.'
projectsAll['tier'] = projectsAll.bandwidth_limit.apply(lambda x: 'free' if x <=50 else 'paid')


In [7]:
print(len(projectsAll),len(projectsAll[projectsAll.tier=='free'].user_id.unique()))

16436 6799


In [8]:
freeProjects = projectsAll[projectsAll.tier=='free']

In [9]:
#Ensure the interval_start columns are of type datetime
bbr.interval_start = pd.to_datetime(bbr.interval_start)
bst.interval_start = pd.to_datetime(bst.interval_start)

In [10]:
#Check the call to dt.month returns expected values before using.
print(type(bbr.interval_start.dt.month[0]))
print(bbr.interval_start.dt.month)

<class 'numpy.int64'>
0          6
1          4
2          4
3          4
4          4
          ..
1013359    3
1013360    3
1013361    3
1013362    2
1013363    2
Name: interval_start, Length: 1013364, dtype: int64


In [11]:
#Cut bbr and bst off at end of April to match the other data
bbr = bbr[bbr.interval_start<dt.datetime(2021,5,1,0,0,0)]
bst = bst[bst.interval_start<dt.datetime(2021,5,1,0,0,0)]

In [12]:
#Get user usage data for the April period so we can compare to what node operators earned in that period
#(Someone can't profit from the system if they pay more than they earn)
bbrApril = bbr[(bbr['interval_start'].dt.year==2021) & (bbr['interval_start'].dt.month==4)]
bstApril = bst[(bst['interval_start'].dt.year==2021) & (bst['interval_start'].dt.month==4)]

In [13]:
#Ben confirmed in office hours that 'settled' was the correct field to aggregate on for bandwidth usage
user_bw_April = bbr.groupby('project_id',as_index=False).aggregate({'settled':'sum'})

In [14]:
#Ben confirmed in ofc hrs that 'remote' was correct field to aggregate on for storage usage
bst = bst.sort_values(by='interval_start', ascending=False).reset_index(drop=True)
user_storage_April = bst.groupby('project_id',as_index=False).aggregate({'remote':'first', 'interval_start':'first'})
user_storage_April

,project_id,remote,interval_start
0,00070a32ce6f412787daba6258ea3c9e,1833334528,2021-04-30 23:12:00
1,0033465c09d34eccb9acdd87ca973469,3585792,2021-04-21 04:32:00
2,0034d6dfb7aa4141bc2acb94966f28f8,571648,2021-04-30 15:50:00
3,0035a2f535c740e0b0284ac41575cf05,32005985024,2021-04-30 15:50:00
4,004a41b726d941abb521a69824884b5e,629896704,2021-04-30 23:12:00
...,...,...,...
3091,ff766075af264655843907aaefd3a37e,29696,2021-04-30 23:12:00
3092,ffa8d6d238804a00a21b816e1ffa9b12,0,2021-04-30 23:12:00
3093,ffaa7e31dd1640779dc0fa410f6e1037,11819008,2021-04-30 23:12:00
3094,ffd6a82ab1ec41c09e8a1b8c40475459,376018176,2021-04-30 16:07:00


In [15]:
#convert to gb
user_bw_April.settled = user_bw_April.settled/1000000000

In [16]:
user_storage_April.remote = user_storage_April.remote/1000000000

In [17]:
user_bw_April.rename(columns={'settled':'user_bw_April'},inplace=True)


In [18]:
user_storage_April.rename(columns={'remote':'user_storage_April'},inplace=True)
user_storage_April = user_storage_April.drop(columns={'interval_start'},axis=1)
user_storage_April

,project_id,user_storage_April
0,00070a32ce6f412787daba6258ea3c9e,1.833335
1,0033465c09d34eccb9acdd87ca973469,0.003586
2,0034d6dfb7aa4141bc2acb94966f28f8,0.000572
3,0035a2f535c740e0b0284ac41575cf05,32.005985
4,004a41b726d941abb521a69824884b5e,0.629897
...,...,...
3091,ff766075af264655843907aaefd3a37e,0.000030
3092,ffa8d6d238804a00a21b816e1ffa9b12,0.000000
3093,ffaa7e31dd1640779dc0fa410f6e1037,0.011819
3094,ffd6a82ab1ec41c09e8a1b8c40475459,0.376018


In [19]:
#Move the actions (uploads vs. downloads) into separate columns and sum the up by project
bbr['uploads'] = bbr.action==1
bbr['downloads']= bbr.action==2
bbrActivity = bbr.groupby('project_id',as_index=False).aggregate({'uploads':'sum','downloads':'sum'})

In [20]:
bbr.uploads = bbr.uploads.astype(int)
bbr.downloads = bbr.downloads.astype(int)
bbr

,Unnamed: 0,interval_start,action,inline,settled,project_id,uploads,downloads
1,1,2021-04-20 17:00:00,1,0,11766784,a965308845f94c34a312cd34d0e30bc5,1,0
2,2,2021-04-20 17:00:00,2,0,8471040,a965308845f94c34a312cd34d0e30bc5,0,1
3,3,2021-04-20 18:00:00,1,0,562432,a965308845f94c34a312cd34d0e30bc5,1,0
4,4,2021-04-20 18:00:00,2,0,361216,a965308845f94c34a312cd34d0e30bc5,0,1
5,5,2021-04-20 19:00:00,1,0,27832320,a965308845f94c34a312cd34d0e30bc5,1,0
...,...,...,...,...,...,...,...,...
1013359,237936,2021-03-10 13:00:00,2,2451,0,99b54f4665b441269c87660036f8e42e,0,1
1013360,237937,2021-03-10 15:00:00,2,0,75193856,99b54f4665b441269c87660036f8e42e,0,1
1013361,237938,2021-03-10 16:00:00,2,0,2381824,99b54f4665b441269c87660036f8e42e,0,1
1013362,237939,2021-02-12 22:00:00,2,0,0,d346180ab62e4142aeb0d34a31fdd382,0,1


In [21]:
#Create a column that indicates the month and year corresponding to each upload, download event
bbr['period'] = bbr.interval_start.dt.to_period('M')

In [22]:
#Get last 6 months worth of uploads downloads only
uploadAmounts = bbr[(bbr.uploads==1) & (bbr.period>'2020-10') & (bbr.period<'2021-05')].groupby(['project_id','period'])['settled'].sum()
#uploadAmounts = uploadAmounts.rename(columns={'settled':'uploadAmount'})
downloadAmounts = bbr[(bbr.downloads==1) & (bbr.period>'2020-10') & (bbr.period<'2021-05')].groupby(['project_id','period'])['settled'].sum()
#downloadAmounts = downloadAmounts.rename(columns={'settled':'downloadAmount'})

In [23]:
#Reshape each month of downloads and uploads into a separate column
uploadsPP = uploadAmounts.to_frame().unstack().fillna(0)
uploadsPP.columns = uploadsPP.columns.droplevel(0)
downloadsPP = downloadAmounts.to_frame().unstack().fillna(0)
downloadsPP.columns = downloadsPP.columns.droplevel(0)


In [24]:
#Rename those new columns
uploadsPP = uploadsPP.reset_index()
uploadsPP.columns = uploadsPP.columns.astype(str)
uploadsPP.rename(columns={'2020-11':'uploads-2020-11','2020-12':'uploads-2020-12','2021-01':'uploads-2021-01',
                          '2021-02':'uploads-2021-02','2021-03':'uploads-2021-03','2021-04':'uploads-2021-04'}
                 ,inplace=True)
downloadsPP.columns = downloadsPP.columns.astype(str)
downloadsPP.rename(columns={'2020-11':'downloads-2020-11','2020-12':'downloads-2020-12','2021-01':'downloads-2021-01',
                          '2021-02':'downloads-2021-02','2021-03':'downloads-2021-03','2021-04':'downloads-2021-04'}
                 ,inplace=True)

In [25]:
print(len(bbr.project_id.unique()), len(uploadsPP.project_id.unique()))

3048 2603


**You get lots of NaNs introduced here, since the number of rows that have uploads is less than the total number of rows. Need to fill with 0s**

In [26]:
#Get upload intervals
bbrActivityUploads = bbr[bbr.uploads==1]
bbrActivityUploads = bbrActivityUploads.groupby('project_id')['interval_start'].apply(list).reset_index(name='upload_intervals')
bbrActivityUploads = bbrActivityUploads.merge(uploadsPP, left_on='project_id',right_on='project_id',how='left').fillna(0)
bbrActivityUploads

,project_id,upload_intervals,uploads-2020-11,uploads-2020-12,uploads-2021-01,uploads-2021-02,uploads-2021-03,uploads-2021-04
0,00070a32ce6f412787daba6258ea3c9e,"[2020-11-22 18:00:00, 2020-11-22 19:00:00, 202...",1.276503e+11,6.133135e+08,0.0,1.475253e+09,0.0,0.0
1,0033465c09d34eccb9acdd87ca973469,"[2021-04-20 08:00:00, 2021-04-20 10:00:00, 202...",0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,13386496.0
2,0034d6dfb7aa4141bc2acb94966f28f8,"[2021-04-28 22:00:00, 2021-04-29 00:00:00, 202...",0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,2157312.0
3,0035a2f535c740e0b0284ac41575cf05,"[2020-11-17 15:00:00, 2020-11-17 16:00:00, 202...",1.611884e+11,0.000000e+00,0.0,0.000000e+00,0.0,0.0
4,004f3f59dd8047e598a28cb2519f821b,"[2020-12-02 18:00:00, 2020-12-02 19:00:00, 202...",0.000000e+00,5.687355e+09,0.0,0.000000e+00,0.0,0.0
...,...,...,...,...,...,...,...,...
3034,ffa8d6d238804a00a21b816e1ffa9b12,"[2021-04-22 11:00:00, 2021-04-22 13:00:00, 202...",0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,2039808.0
3035,ffaa7e31dd1640779dc0fa410f6e1037,"[2021-04-20 12:00:00, 2021-04-20 13:00:00, 202...",0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,44228608.0
3036,ffc51149ffa1465f84825775af6a397e,"[2020-06-23 16:00:00, 2020-06-23 17:00:00, 202...",0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0
3037,ffd6a82ab1ec41c09e8a1b8c40475459,"[2020-10-07 02:00:00, 2020-10-07 03:00:00, 202...",0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.0,0.0


In [27]:
#Get download intervals
bbrActivityDownloads = bbr[bbr.downloads==1]
bbrActivityDownloads = bbrActivityDownloads.groupby('project_id')['interval_start'].apply(list).reset_index(name='download_intervals')
bbrActivityDownloads = bbrActivityDownloads.merge(downloadsPP, left_on='project_id',right_on='project_id',how='left').fillna(0)
bbrActivityDownloads

,project_id,download_intervals,downloads-2020-11,downloads-2020-12,downloads-2021-01,downloads-2021-02,downloads-2021-03,downloads-2021-04
0,00070a32ce6f412787daba6258ea3c9e,"[2020-11-22 18:00:00, 2020-11-22 20:00:00, 202...",2.080798e+09,0.0,0.0,1.043267e+09,0.0,0.0
1,0034d6dfb7aa4141bc2acb94966f28f8,"[2021-04-28 22:00:00, 2021-04-29 00:00:00, 202...",0.000000e+00,0.0,0.0,0.000000e+00,0.0,1243904.0
2,00616483b59c47b19e864329614f783f,"[2020-06-29 02:00:00, 2020-06-29 03:00:00, 202...",0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0
3,006de9e8ca6e4b9290b36771b9885e2f,"[2021-04-24 11:00:00, 2021-04-24 13:00:00, 202...",0.000000e+00,0.0,0.0,0.000000e+00,0.0,19133440.0
4,008c2211812a4905bcfc5e54f9a4648a,[2020-11-06 15:00:00],0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0
...,...,...,...,...,...,...,...,...
1882,ff955d5923b94553ac464951502902c8,"[2020-09-25 09:00:00, 2020-09-26 08:00:00, 202...",1.870148e+10,0.0,0.0,0.000000e+00,0.0,0.0
1883,ffaa7e31dd1640779dc0fa410f6e1037,"[2021-04-20 12:00:00, 2021-04-20 14:00:00]",0.000000e+00,0.0,0.0,0.000000e+00,0.0,445184.0
1884,ffc51149ffa1465f84825775af6a397e,"[2020-06-23 16:00:00, 2020-06-23 17:00:00, 202...",0.000000e+00,0.0,0.0,0.000000e+00,0.0,0.0
1885,ffd6a82ab1ec41c09e8a1b8c40475459,"[2021-03-28 11:00:00, 2021-03-28 13:00:00, 202...",0.000000e+00,0.0,0.0,0.000000e+00,343699712.0,0.0


In [28]:
#merge together the upload and download interval lists
bbrActivity2 = bbrActivityUploads.merge(bbrActivityDownloads, left_on='project_id',right_on='project_id',how='outer').fillna(0)

In [29]:
#then merge with overall upload and download counts
bbrActivity = bbrActivity.merge(bbrActivity2,left_on='project_id',right_on='project_id',how='outer')

In [30]:
#Check for nans
bbrActivity.isna().sum().sum()

0

In [31]:
#Add in the April user costs
bbrActivity = bbrActivity.merge(user_storage_April, left_on='project_id',right_on='project_id',how='left').fillna(0)
bbrActivity = bbrActivity.merge(user_bw_April, left_on='project_id',right_on='project_id',how='left').fillna(0)

In [32]:
#check for nans
bbrActivity.isna().sum().sum()

0

**At his point we're merging all project data with data on active projects, so we're going to end up with a lot of NaNs for projects that haven't been active**

In [33]:
mapProjToNode = freeProjects[['created_at','project_id','user_id','tier']].merge(bbrActivity,left_on='project_id',right_on='project_id',how='left').fillna(0)
print(len(freeProjects.project_id.unique()),len(bbrActivity),len(mapProjToNode))

7410 3048 7410


In [34]:
mapProjToNode[mapProjToNode.user_bw_April == mapProjToNode.user_bw_April.max()]

,created_at,project_id,user_id,tier,uploads,downloads,upload_intervals,uploads-2020-11,uploads-2020-12,uploads-2021-01,...,uploads-2021-04,download_intervals,downloads-2020-11,downloads-2020-12,downloads-2021-01,downloads-2021-02,downloads-2021-03,downloads-2021-04,user_storage_April,user_bw_April
3601,2021-04-20,9b00834f37834f66830f42a058fea7a6,f649af7a489f4edda2a2bbdbbfc39353,free,97.0,11.0,"[2021-04-21 05:00:00, 2021-04-21 06:00:00, 202...",0.0,0.0,0.0,...,2.115748e+12,"[2021-04-21 05:00:00, 2021-04-22 04:00:00, 202...",0.0,0.0,0.0,0.0,0.0,1.395743e+09,492.638362,2117.143406


In [35]:
#from project file -- with bandwidth-limit converted to GB
projectsAll[projectsAll.project_id=='9b00834f37834f66830f42a058fea7a6']

,Unnamed: 0,created_at,project_id,user_id,usage_limit,bandwidth_limit,tier
7068,4393,2021-04-20,9b00834f37834f66830f42a058fea7a6,f649af7a489f4edda2a2bbdbbfc39353,2000000000000,50.0,free


In [36]:
bbr[(bbr.project_id=='9b00834f37834f66830f42a058fea7a6') & (bbr.period == '2021-04')].settled.sum()/1000000000

2117.143406336

In [37]:
bst[(bst.project_id=='9b00834f37834f66830f42a058fea7a6') & (bbr.period == '2021-04')].remote.max()/1000000000

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


492.638362112

**The fact that there are significantly more projects than we have project activity for isn't a concern. 
Ben wrote in Slack: 'bbr project ID's may not exhaust all ID's in the projects tables because we have plenty of customers who have created accounts and projects but actually haven't uploaded anything or transacted with the network in any other way. so these ID's would appear in projects but not in bbr or bst'**

In [38]:
#replace NaN values with zeros in the uploads and downloads columns
mapProjToNode.uploads = mapProjToNode.uploads.replace(np.nan, 0)
mapProjToNode.downloads = mapProjToNode.downloads.replace(np.nan, 0)
cols = ['uploads-2020-11','uploads-2020-12','uploads-2021-01','uploads-2021-02','uploads-2021-03',
        'uploads-2021-04','downloads-2020-11','downloads-2020-12','downloads-2021-01','downloads-2021-02',
        'downloads-2021-03','downloads-2021-04','user_storage_April','user_bw_April']
mapProjToNode[cols] = mapProjToNode[cols].replace(np.nan,0)
mapProjToNode

,created_at,project_id,user_id,tier,uploads,downloads,upload_intervals,uploads-2020-11,uploads-2020-12,uploads-2021-01,...,uploads-2021-04,download_intervals,downloads-2020-11,downloads-2020-12,downloads-2021-01,downloads-2021-02,downloads-2021-03,downloads-2021-04,user_storage_April,user_bw_April
0,2021-04-21,0044787a0dd54a6eaf0f71f85fe9b751,0319b622f638494cae46937101ed1f70,free,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-05-21,0044d8c953ad44b183eb5f2e146f0749,1dac3d07be024def813b0f039f9fb781,free,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-05-18,004f6495f57f48dfba69da9cff8d0b5e,ab98cd37fefd47d48e071e6fa26d382b,free,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-05-02,00dd00740a1c43b1a34e3c09894a67c8,4ea94fd059dc4f8a8a1e24defc7fae39,free,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-04-20,00f014b807cf4d95a25a635c7a0bb105,28152efd812946029306710c711c8d52,free,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,2021-04-13,ff3c1daab9294695b15f636eaa519886,7c8acf7a7037486dbf9a7a9a76489c89,free,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7406,2021-05-07,ffc333a2e058499690e084fc1611f13c,2fde122d63264ad09291a8f053f1156c,free,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7407,2021-04-24,ffc3ac8a37774bfc9a68cce81ecb0707,a7609949d3054b78ab2d0379a3bf7e39,free,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7408,2021-05-24,ffc45d28750b4993b2b2f64a65eaf68b,f283eef8ba044b9884eef89ece8042f1,free,0.0,0.0,0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
#get the upload and download aggregates across projects for each user
mapProjToNodeByUser1 = mapProjToNode.groupby('user_id').aggregate({'downloads':'sum','uploads':'sum',
                                                                  'user_storage_April':'sum',
                                                                   'user_bw_April':'sum',
                                                                   'uploads-2020-11':'sum','uploads-2020-12':'sum',
                                                                   'uploads-2021-01':'sum','uploads-2021-02':'sum',
                                                                   'uploads-2021-03':'sum','uploads-2021-04':'sum',
                                                                  'downloads-2020-11':'sum','downloads-2020-12':'sum',
                                                                   'downloads-2021-01':'sum','downloads-2021-02':'sum',
                                                                   'downloads-2021-03':'sum','downloads-2021-04':'sum'})

In [40]:
#get the aggregated created_at dates
mapProjToNodeByUser2 = mapProjToNode.groupby('user_id')['created_at'].unique().apply(list).reset_index(name='project_creation_dates')

**Need to fix the upload- and download-interval code below to exclude 0s from the intervals as well, not just 'nan's**

In [41]:
#get the aggregated upload intervals
mapProjToNodeByUser3 = mapProjToNode.groupby('user_id')['upload_intervals'].apply(list).reset_index()
mapProjToNodeByUser3.upload_intervals = mapProjToNodeByUser3.upload_intervals.apply(lambda x: [i for i in x if str(i) != "nan"])

In [42]:
#get the aggregated download intervals
mapProjToNodeByUser4 = mapProjToNode.groupby('user_id')['download_intervals'].apply(list).reset_index()
mapProjToNodeByUser4.download_intervals = mapProjToNodeByUser4.download_intervals.apply(lambda x: [i for i in x if str(i) != "nan"])

In [43]:
#put all of the pieces together for a per-user view and check lengths
mapProjToNodeByUser = mapProjToNodeByUser1.merge(mapProjToNodeByUser2,left_on='user_id',right_on='user_id',how ='outer')
print(len(mapProjToNodeByUser1), len(mapProjToNodeByUser2),len(mapProjToNodeByUser))
mapProjToNodeByUser = mapProjToNodeByUser.merge(mapProjToNodeByUser3,left_on='user_id',right_on='user_id',how ='outer')
print(len(mapProjToNodeByUser), len(mapProjToNodeByUser3))
mapProjToNodeByUser = mapProjToNodeByUser.merge(mapProjToNodeByUser4,left_on='user_id',right_on='user_id',how ='outer')
print(len(mapProjToNodeByUser), len(mapProjToNodeByUser4))

6799 6799 6799
6799 6799
6799 6799


In [44]:
#add the free vs. paid tier column
mapProjToNodeByUser = mapProjToNodeByUser.merge(freeProjects[['user_id','tier']],left_on='user_id',right_on='user_id',how='left')

In [45]:
#map in the email_hashes so we can get the node addresses
mapProjToNodeByUser = mapProjToNodeByUser.merge(usersAll[['id','email_hash']],left_on='user_id',right_on='id',how='left')

In [46]:
#drop the extra column
mapProjToNodeByUser= mapProjToNodeByUser.drop('id',axis=1)
mapProjToNodeByUser

,user_id,downloads,uploads,user_storage_April,user_bw_April,uploads-2020-11,uploads-2020-12,uploads-2021-01,uploads-2021-02,uploads-2021-03,...,downloads-2020-12,downloads-2021-01,downloads-2021-02,downloads-2021-03,downloads-2021-04,project_creation_dates,upload_intervals,download_intervals,tier,email_hash
0,004596a1c8bd4d899d1db6a78df12f05,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,[2021-05-08],[0],[0],free,5558b9f30f1a009cacb00009906f4a61c68345011315fb...
1,0074a096b4c349b0a8aeee959101cbba,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,[2021-05-09],[0],[0],free,a592d85b449dc3efb6365dbb5a97ea3d27cb2213be1edd...
2,00766f132d11461eb6885f7543bc3a83,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,[2021-04-20],[0],[0],free,638185fc42ff9fc17669071cd3544f8235426405b5f16e...
3,007807b3e1304059a9571b143e0664d6,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,[2021-05-20],[0],[0],free,6b0964f25218f89346c2950c6175e113c3498be7a09b46...
4,00b2baff8af14bea9d5808455edb9768,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,[2021-04-20],[0],[0],free,dae7216070d1a9a609202fbcc6534dc8a444f53cde162d...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7405,ffda7a23b31243508b1ff193d0aafe7c,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,[2021-04-23],[0],[0],free,202777359bafd830e97dd8b1b5a0359adba6cbbc17816a...
7406,ffdd417180604bbea3f6ed7443a2b209,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,[2021-05-04],[0],[0],free,027d265cf2fd2d7bb66a3e5b6b44b4878da31158915de5...
7407,ffe460a7367e42ebbc4399d7f451521d,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,[2021-05-06],[0],[0],free,8ce0a91beec69e8b6f67aee63b02bd6aeef664e1bf4ec2...
7408,ffeff75038df4f08b516695e539682be,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,[2021-05-05],[0],[0],free,4f5fa648c0d512ed4dc3760863184b4950d8ecbb4600e4...


In [47]:
#There are 2 NaNs in the df at this point; they're both missing email_hashes. The corresponding user ids don't
#appear in the usersAll data set ...
mapProjToNodeByUser.isnull().sum().sum()

0

In [48]:
mapProjToNodeByUser[mapProjToNodeByUser.email_hash.isna()]

,user_id,downloads,uploads,user_storage_April,user_bw_April,uploads-2020-11,uploads-2020-12,uploads-2021-01,uploads-2021-02,uploads-2021-03,...,downloads-2020-12,downloads-2021-01,downloads-2021-02,downloads-2021-03,downloads-2021-04,project_creation_dates,upload_intervals,download_intervals,tier,email_hash


In [49]:
print(len(mapProjToNodeByUser.drop_duplicates(subset=['user_id','downloads','uploads','user_storage_April',
       'user_bw_April', 'uploads-2020-11',
       'uploads-2020-12', 'uploads-2021-01', 'uploads-2021-02',
       'uploads-2021-03', 'uploads-2021-04', 'downloads-2020-11',
       'downloads-2020-12', 'downloads-2021-01', 'downloads-2021-02',
       'downloads-2021-03', 'downloads-2021-04','tier', 'email_hash'])), len(mapProjToNodeByUser.user_id.unique()))

6799 6799


In [50]:
mapProjToNodeByUser = mapProjToNodeByUser.drop_duplicates(subset=['user_id','downloads','uploads','user_storage_April',
       'user_bw_April', 'uploads-2020-11',
       'uploads-2020-12', 'uploads-2021-01', 'uploads-2021-02',
       'uploads-2021-03', 'uploads-2021-04', 'downloads-2020-11',
       'downloads-2020-12', 'downloads-2021-01', 'downloads-2021-02',
       'downloads-2021-03', 'downloads-2021-04','tier', 'email_hash'])

In [51]:
mapProjToNodeByUser.to_pickle('userDataFree.pkl')